In [8]:
# 所有的头

In [125]:
import torch # for save
import os
from van import Fan
from localconfig import config
import datetime
import random
from pprint import pprint
import html
from pyltp import SentenceSplitter
from pyltp import Postagger
from pyltp import Segmentor
from pyltp import NamedEntityRecognizer
import matplotlib.pyplot as plt
from tqdm import tqdm
from IPython.display import Image
from IPython.core.display import HTML 
import pyecharts
from pyecharts import Pie
from pyecharts import Line

%matplotlib inline

plt.rcParams['font.sans-serif']=['FangSong'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号


In [10]:
# 所有的配置

In [11]:
keyword = '清华大学' # 关注的关键词
until_year = 2000 # 最早的年份
until_num = 10000 # 截止的饭否数量
timeout = (60,60) # 超时时间

In [12]:
# 登录
fan = Fan(config['Consumer key'], config['Consumer secret'])
fan.xauth(config['my username'], config['my password'])

{'oauth_token': '1405515-071ad3666b5af9441d5cbb0dc1d3985b',
 'oauth_token_secret': '43cc293f5d127423f960b8636688577b'}

In [13]:
# 本小时内可用API次数
fan.request('GET', 'account/rate_limit_status', {}, timeout=timeout)

{'reset_time': 'Mon Mar 18 07:26:11 +0000 2019',
 'remaining_hits': 150,
 'hourly_limit': 150,
 'reset_time_in_seconds': 1552893971}

In [14]:
# 采集关键词消息
def datetimefromstr(s):
    return datetime.datetime.strptime(s, '%a %b %d %H:%M:%S %z %Y')
def get_status(keyword, until_year, until_num):
    latest = fan.request('GET', 'search/public_timeline', {'q':keyword, 'count':60}, timeout=timeout)
    while True:
        year = datetimefromstr(latest[-1]['created_at']).year
        if len(latest) >= until_num:
            break
        if year < until_year:
            break
        print(len(latest), year,'/', until_num, until_year)
        latest += fan.request('GET', 'search/public_timeline', { 'q':keyword, 'count':60, 'max_id':latest[-1]['id']}, timeout=timeout)[1:]
    return latest

In [15]:
# 下载并暂存
statuses = get_status(keyword, until_year, until_num)

60 2018 / 10000 2000
119 2018 / 10000 2000
178 2018 / 10000 2000
237 2018 / 10000 2000
296 2018 / 10000 2000
355 2017 / 10000 2000
414 2017 / 10000 2000
473 2017 / 10000 2000
532 2017 / 10000 2000
591 2016 / 10000 2000
650 2016 / 10000 2000
709 2016 / 10000 2000
768 2015 / 10000 2000
827 2015 / 10000 2000
886 2015 / 10000 2000
945 2014 / 10000 2000
1004 2014 / 10000 2000
1063 2014 / 10000 2000
1122 2014 / 10000 2000
1181 2014 / 10000 2000
1240 2013 / 10000 2000
1299 2013 / 10000 2000
1358 2013 / 10000 2000
1417 2013 / 10000 2000
1476 2013 / 10000 2000
1535 2013 / 10000 2000
1594 2013 / 10000 2000
1653 2013 / 10000 2000
1712 2013 / 10000 2000
1771 2013 / 10000 2000
1830 2013 / 10000 2000
1889 2013 / 10000 2000
1948 2013 / 10000 2000
2007 2013 / 10000 2000
2066 2013 / 10000 2000
2125 2013 / 10000 2000
2184 2013 / 10000 2000
2243 2013 / 10000 2000
2302 2013 / 10000 2000
2361 2013 / 10000 2000
2420 2013 / 10000 2000
2479 2013 / 10000 2000
2538 2013 / 10000 2000
2597 2013 / 10000 2000
2656 

In [16]:
torch.save(statuses,'tmp_statuses.torch.pkl')

In [ ]:
# 恢复
# statuses = torch.load('tmp_statuses.torch.pkl')

In [17]:
print(len(statuses))

10031


In [170]:
# common
def get_top(l, num=5, reverse=False): # 从大到小
    cnt = {}
    for x in l:
        if type(x) == tuple and len(x) == 3:
            x = x[0]
        if x not in cnt:
            cnt[x] = 0
        cnt[x] += 1
    cnt_list = [(k,cnt[k],cnt[k]/len(l)) for k in cnt]
    cnt_list = sorted(cnt_list, key=lambda x:x[1])[::-1]
    if reverse:
        cnt_list = cnt_list[::-1]
    if num == -1:
        return cnt_list
    return cnt_list[:num]
def huabing(y): # x [(label,,0.xxx)]
    attr = [x[0] for x in y]
    v1 = [x[2] for x in y]
    if sum(v1) < 1.0:
        attr.append('其他')
        v1.append(1.0 - sum(v1))
    pie = Pie('')
    pie.add('', attr, v1, is_label_show=True)
    return pie

In [21]:
# 从消息关系本身入手

In [22]:
# 时间段分布

In [29]:
print(datetimefromstr(statuses[-1]['created_at']))
print(datetimefromstr(statuses[0]['created_at']))
begin_year = datetimefromstr(statuses[-1]['created_at']).year
end_year = datetimefromstr(statuses[0]['created_at']).year

2011-07-01 13:24:12+00:00
2019-03-13 09:32:04+00:00


In [138]:
huabing(get_top([datetimefromstr(x['created_at']).year for x in statuses], num = -1))

In [139]:
arr = sorted(get_top([datetimefromstr(x['created_at']).year for x in statuses],num=-1),key = lambda x:x[0])
attr = [str(x[0]) for x in arr ]
v1 = [x[1] for x in arr ]
line = Line('')
line.add('清华大学', attr, v1,is_smooth=True,is_label_show=True,mark_line=[ "average"])
# line.add("商家B", attr, v2, is_smooth=True, mark_line=["max", "average"])
line

In [32]:
# 用户信息文件
os.path.exists('./users.torch.pkl')

True

In [33]:
# 预处理用户信息
userid_to_info = {}
if os.path.exists('./users.torch.pkl'):
    userid_to_info = torch.load('users.torch.pkl')
for x in statuses:
    userid_to_info[x['user']['id']] = x['user']
torch.save(userid_to_info, 'users.torch.pkl')
print(len(userid_to_info))

3265


In [34]:
# 发表人数/总消息数量
uni_num = len(set([x['user']['id'] for x in statuses ]))
print(uni_num, len(statuses), uni_num/len(statuses))

3076 10031 0.3066493869006081


In [184]:
# 发表top30的人
users_top = get_top([x['user']['id'] for x in statuses], num=30)
names_top = [ (userid_to_info[x[0]]['name'] , x[0], x[1], x[2]) for x in users_top]
pprint(names_top)
huabing(names_top)

[('61F_Kevin', '~GZOJ7LF5RO4', 554, 0.0552287907486791),
 ('积极向下', 'yuanbo_thu', 483, 0.04815073272854152),
 ('野孩子奇妙旅', 'ixinqi', 481, 0.04795135081248131),
 ('水木小白', 'xiaobaishu', 477, 0.047552586980360884),
 ('momokaqiQ', '~EyBuHsVnfxY', 372, 0.037085036387199684),
 ('Wishee', 'wishee', 210, 0.0209351011863224),
 ('木子思想', '~FBpqUfTy9KA', 189, 0.01884159106769016),
 ('财经频道', 'cashmoney', 150, 0.014953643704516),
 ('heaven0701', 'heaven0701', 147, 0.01465457083042568),
 ('万物初始之风', '~esDaRQrrXyQ', 143, 0.014255806998305254),
 ('孟庆迪', '~lSMLG0nb2-I', 113, 0.011265078257402054),
 ('dylanhaha', '~Yz1WTzJcs-8', 104, 0.010367859635131094),
 ('少华LEO', '~ghujSq7_VJs', 87, 0.00867311334861928),
 ('fish_y007', 'fish_y007', 71, 0.007078058020137574),
 ('Manchild.mpg', 'manchild09', 65, 0.0064799122719569335),
 ('sunflowerzzz', 'Sunflowerzzz', 61, 0.006081148439836507),
 ('星雲之外', '~nTE0DS2-Kis', 50, 0.004984547901505334),
 ('hcdkaka', '~ycoz5gaPq1s', 47, 0.0046854750274150135),
 ('siaosin', 'siaos

In [142]:
def show_user_info(x):
    user = userid_to_info[x]
    pprint(user)
    return Image(url = user['profile_image_url_large'])

In [151]:
show_user_info(users_top[5][0])

{'birthday': '1991-01-29',
 'created_at': 'Fri Nov 26 04:05:56 +0000 2010',
 'description': '',
 'favourites_count': 0,
 'followers_count': 46,
 'following': False,
 'friends_count': 38,
 'gender': '女',
 'id': 'wishee',
 'location': '北京',
 'name': 'Wishee',
 'notifications': False,
 'photo_count': 51,
 'profile_background_color': '#acdae5',
 'profile_background_image_url': 'https://s3.meituan.net/v1/mss_3d027b52ec5a4d589e68050845611e68/avatar/b0/00/jy/y1_1291285105.jpg?1291285105',
 'profile_background_tile': True,
 'profile_image_url': 'http://s3.meituan.net/v1/mss_3d027b52ec5a4d589e68050845611e68/avatar/s0/00/jy/y1.jpg?1291872033',
 'profile_image_url_large': 'http://s3.meituan.net/v1/mss_3d027b52ec5a4d589e68050845611e68/avatar/l0/00/jy/y1.jpg?1291872033',
 'profile_link_color': '#0066cc',
 'profile_sidebar_border_color': '#b2d1a3',
 'profile_sidebar_fill_color': '#e2f2da',
 'profile_text_color': '#222222',
 'protected': False,
 'screen_name': 'Wishee',
 'statuses_count': 1476,
 'uni

In [180]:
# 被引用top20的消息

repost_top = get_top([x['repost_status']['id'] for x in statuses if 'repost_status' in x], num=20)
for i in range(len(repost_top)):
    x = repost_top[i]
    for y in statuses:
        if 'repost_status' in y and y['repost_status']['id'] == x[0]:
            repost_top[i] = ('id: '+y['repost_status']['user']['id'] + ' name: '+y['repost_status']['user']['name']  +' '+ y['repost_status']['text'],x[0], x[1],x[2])
            break
            
pprint(repost_top)

[('id: wangxing name: 王兴 '
  '清华从2017年起全面推行大类招生培养和管理，纳入本科招生的所有专业将整合为16大类。《清华大学章程》描述的人才培养目标是：健全人格、宽厚基础、创新思维、全球视野和社会责任感。',
  'fOxn693eMJ8',
  65,
  0.09601181683899557),
 ('id: bisons name: 毒艹 '
  '昨天才知道。清华大学对各省的招生人数，是按照当年庚子赔款比例分配的。当年，给中央一下子拿不出来那么多钱，由各省分摊来赔。赔款多，人数就多。后面清华也一直想改，但是改来改去总拿不出一个让所有人满意的方案。',
  'ooE5yvNnXXE',
  24,
  0.03545051698670606),
 ('id: wangxing name: 王兴 清华大学2018年的预算是269亿。',
  'byJ0XgSUbHo',
  22,
  0.03249630723781388),
 ('id: gulan name: 野小合 '
  '最近据说说雄安的事缓下来了，清华大学雄安新区规划建设发展高峰论坛上一位教授作了报告，网上有，内容很多，其中一条，雄安地势很低洼，一旦有洪水， '
  '这地方一定被淹。上千年来那地方都没有形成过城市，这不是没原因的。',
  'J2qlqnWtAI8',
  22,
  0.03249630723781388),
 ('id: zuoxiaozuzhou name: 左小祖咒 '
  '【超级大国】清华大学当代国际关系研究院院长阎学通称，中国已经与美国并称超级大国。。。网友戏称：污染超级大国，腐败超级大国，有毒食品超级大国，失学儿童超级大国，维稳开支超级大国，三公支出超级大国，官员数量超级大国，上访人数超级大国，对外移民超级大国。。。',
  'VcATiZAmjkU',
  18,
  0.026587887740029542),
 ('id: riair name: 小日. '
  '在资源宾馆向北不远处，清华大学校园内，王兴读大三，刚建了人生中第一个网站。他喜欢跳一种叫“黄土黄”的传统舞蹈，跳时赤裸上身，胸前绑鼓。［看到某篇写2000年跨年当时一些大佬的众生相里面对王村长的描写',
  'WPZh1EfL4vw',
  14,
  0.020

In [185]:
# 引用top20的人
repost_user_id_top = get_top([x['repost_status']['user']['id'] for x in statuses if 'repost_status' in x], num=20)
repost_name_top =  [ (userid_to_info[x[0]]['name'] , x[0], x[1], x[2]) for x in repost_user_id_top]
pprint(repost_name_top)

[('王兴', 'wangxing', 98, 0.1447562776957164),
 ('咬指头', '~DYFfhxVCIRI', 25, 0.03692762186115214),
 ('野小合', 'gulan', 25, 0.03692762186115214),
 ('毒艹', 'bisons', 24, 0.03545051698670606),
 ('剑花', 'dezhaos', 19, 0.028064992614475627),
 ('左小祖咒', 'zuoxiaozuzhou', 18, 0.026587887740029542),
 ('二他姐姐', 'znz28', 16, 0.023633677991137372),
 ('财经频道', 'cashmoney', 14, 0.0206794682422452),
 ('拧巴人儿', 'mehitabel', 14, 0.0206794682422452),
 ('左堂堂', 'bb0813', 14, 0.0206794682422452),
 ('小日.', 'riair', 14, 0.0206794682422452),
 ('罗妮zyzyyuan', 'zyzyzhangyuan', 14, 0.0206794682422452),
 ('狂乱桂', 'luxiaojie1991', 13, 0.019202363367799114),
 ('楚恻', 'blurmoon', 13, 0.019202363367799114),
 ('超級小饅頭', 'kristy_xiao', 12, 0.01772525849335303),
 ('AlienInvader', 'dcn01', 11, 0.01624815361890694),
 ('暖暖小草莓', '清斯忘', 11, 0.01624815361890694),
 ('石说', 'szshitou', 10, 0.014771048744460856),
 ('无聊图3号机', '~vB7sBb_YWyI', 10, 0.014771048744460856),
 ('淮月V', 'qhgy', 9, 0.013293943870014771)]


In [186]:
# top20 gender
gender_top = get_top([x['user']['gender'] for x in statuses ], num=20)
pprint(gender_top)
huabing(gender_top)

[('', 4624, 0.4609709899312132),
 ('男', 4066, 0.4053434353504137),
 ('女', 1341, 0.13368557471837306)]


In [187]:
# top10 location
location_top10 = get_top
location_top10 = get_top([x['location'].split()[0] for x in statuses if len(x['location'].split())>=1], num=10)
pprint(location_top10)
huabing(location_top10)

[('39.999009,116.324791', 1049, 0.1187995469988675),
 ('北京', 511, 0.057870894677236696),
 ('40.005798,116.328665', 492, 0.05571913929784825),
 ('40.011550,116.328910', 439, 0.04971687429218573),
 ('39.998680,116.324920', 349, 0.03952434881087203),
 ('广东', 239, 0.027066817667044166),
 ('40.002546,116.330195', 206, 0.02332955832389581),
 ('上海', 191, 0.021630804077010193),
 ('40.000187,116.315383', 185, 0.020951302378255945),
 ('40.004751,116.323622', 184, 0.020838052095130236)]


In [188]:
def remove_atag(s):
    if '<a' not in s:
        return s
    start = s.find('>')
    end = s.find('</a>',start)
    return s[start+1:end]

In [189]:
# top20 source
source_top = get_top([remove_atag(x['source']) for x in statuses ], num=20)
pprint(source_top)
huabing(source_top)

[('街旁', 6794, 0.6773003688565448),
 ('网页', 825, 0.082245040374838),
 ('享拍微博通', 453, 0.04516000398763832),
 ('手机上网', 176, 0.017545608613298774),
 ('Android客户端', 171, 0.01704715382314824),
 ('饭否财经频道', 150, 0.014953643704516),
 ('FaWave发微', 133, 0.013258897418004187),
 ('API', 129, 0.012860133585883761),
 ('「御飯 iPhone」', 113, 0.011265078257402054),
 ('FaWave', 101, 0.010068786761040774),
 ('开开Kai', 98, 0.009769713886950453),
 ('御飯 iOS', 93, 0.009271259096799921),
 ('饭否分享', 65, 0.0064799122719569335),
 ('猫饭', 62, 0.006180839397866613),
 ('冒泡', 59, 0.005881766523776293),
 ('FFReeder', 51, 0.00508423885953544),
 ('蘑菇饭', 45, 0.0044860931113548),
 ('尚饭', 35, 0.0034891835310537334),
 ('follow5', 34, 0.003389492573023627),
 ('微博通', 32, 0.0031901106569634133)]


In [190]:
# nlp part

In [152]:
def myfilter(s):
    s = s.replace('<strong>','')
    s = s.replace('</strong>','')
    while '转@' in s:
        pos = s.find('转@')
        end = len(s)
        for endsym in [' ', ']', '：',':','@']:
            end2 = s.find(endsym,pos+2)
            if end2 == -1:
                end2 = len(s)
            end = min(end,end2)
        if end-pos > 15:
            print('alert [转@] too long',s)
            s = s[:pos] + s[pos+2:]
        else:
            s = s[:pos] + s[end:]
    while '@' in s:
        pos = s.find('@')
        end = len(s)
        for endsym in [' ', ']', '：',':','@']:
            end2 = s.find(endsym,pos+1)
            if end2 == -1:
                end2 = len(s)
            end = min(end,end2)
        if end-pos > 15:
            print('alert [@] too long',s)
            s = s[:pos] + s[pos+1:]
        else:
            s = s[:pos] + s[end:]
    if '@' in s:
        print('alert @ in ', s)
    return s

In [153]:
sentences = []
for x in tqdm(statuses):
    s = html.unescape(x['text']) 
    s = myfilter(s)
    if keyword not in s:
        continue
    if '</' in s:
        print(s)
    ss = SentenceSplitter.split(s)
    for t in ss:
        if len(t) > 4:
            sentences.append(t)
    

 10%|▉         | 997/10031 [00:00<00:02, 3412.99it/s]

alert [@] too long 呵呵   @陈心耳浙江，某所高中，800多位高三学子，为最有勇气的五位物理选考勇士加油！（背景：新高考计分规则导致学生们都不爱选物理）（另，某年清华大学发现新招本科生物理基础特别差，后来多个高校联合做了硬规定：以后参加本校自主招生的学生必须选考物理）
alert [@] too long  @陈心耳浙江，某所高中，800多位高三学子，为最有勇气的五位物理选考勇士加油！（背景：新高考计分规则导致学生们都不爱选物理）（另，某年清华大学发现新招本科生物理基础特别差，后来多个高校联合做了硬规定：以后参加本校自主招生的学生必须选考物理）
alert [@] too long @陈心耳浙江，某所高中，800多位高三学子，为最有勇气的五位物理选考勇士加油！（背景：新高考计分规则导致学生们都不爱选物理）（另，某年清华大学发现新招本科生物理基础特别差，后来多个高校联合做了硬规定：以后参加本校自主招生的学生必须选考物理）
alert [@] too long @一条
【教授为保护动植物，买下12000㎡山谷】30年前，台湾清华大学生物教授李家维，为了保护山上的动植物，买下了苗栗县12000㎡山头。2000年，他在山谷中盖了玻璃屋，把收集的化石、文物搬进来，山谷的花房养着各种奇花异卉……整个山林经过几十年保育，逐渐恢复成一片低海拔森林。
alert [转@] too long 转@胡女士通过相亲网站，与在清华大学读书的喀麦隆人卫力相恋。不到2年时间，她支付男友学费、生活费，还在男友说服下卖掉北京的房子，直到发现被骗时，她已向男友账户转账750余万元，部分钱款甚至被卫力转入其喀麦隆怀孕女友的账户。法院以诈骗罪一审判处卫力有期徒刑14年、驱逐出境。
alert [转@] too long  这五条作为对企业管理者的要求也很好。转@王兴《清华大学章程》描述的人才培养目标是：健全人格、宽厚基础、创新思维、全球视野和社会责任感。
alert [@] too long 上午， 称，某校招生组给文理科前十选择北大的考生挨个电话，称北大欺骗他们，不会满足他们专业志愿；@清华大学四川招生组随即回应；北大再称，过去五年砸钱买走的唐某、郭某，需要我讲吗？O北大清华四川招生组互掐北大方面现已删除微博
alert [@] too long @Enjoy雅趣【空气质量监测也能众

 38%|███▊      | 3831/10031 [00:00<00:01, 5684.04it/s]

alert [@] too long 【“强奸陪酒女比强奸良家妇女危害性要小”？】清华大学教授@易延友，在微博上“替李天一的辩护律师说几句”。就法言法，即便有些观点“刺眼”，也是学者言论自由。但辩护词最后一句，说“即便是强奸，强奸陪酒女也比强奸良家妇女危害性小”。教授啊教授，你让我说你什么好呢？@
alert [@] too long 清华大学科学技术与社会研究所蒋劲松副教授、动保网和动保记者沙龙联合创办人、《动物记》主编@动保记者沙龙张丹一起和大家聊聊狗肉节的是与非。   http://talk.weibo.com/ft/201306209748  动保网推出。
alert [@] too long 我在清华大学东大操场：清华超级联赛第三场。4：1胜 老枪。我们成为冬季半程冠军！ 左脚取得开门红 随后以标志性的单骑闯关又进一球。下半场角球开出，@mess354抢前点脚下一蹭进球。最后阶段@... http://jie.pn/p_Ic1ix_YE
alert [转@] too long 周恩来不是挺聪明的么？怎么考不上？ 转@咬指头有个年轻人说，为中华堀起而读书，然后报考清华大学，落榜。就去日本，一所一所的学校考过来，统统落榜。再回国，在刚刚成立的南开大学注册学籍，又遭南开除名处理。干脆去法国，可法国的学校也不认他，不给他文凭。直到他万人之上，功成天下，却仍然不
alert [@] too long 【微访谈：CPI进“1时代” 如何影响你的生活？】感觉身边啥都没便宜，唯一便宜的是股票？9日14：30，腾讯财经联合 ，特邀清华大学金融学教授@songfengming999 ，央视王牌财经评论员兼@cct......  http://url.cn/4wOYR5 b


 79%|███████▊  | 7881/10031 [00:00<00:00, 8927.23it/s]

alert [@] too long 冷静点：清华大学法学博士生@王进文2010在其微博上称，“山东省国土资源厅一把手正在清华大学法学院读博士，但从未见其上课。”清华大学法学院证实，山东省国土资源厅厅长徐景颜确实是该院在读博士生，但他并非是全日制博士生，而是“论文博士”。 厅长不识字，何故乱翻书。
alert [@] too long    ：昨天傍晚洗澡回来的路上，有一对情侣问我近春园宾馆怎么走，我给他们指向的目的地是@清华大学图书馆。希望他们能在知识的海洋中找回迷路的自己，为中华民族的崛起努力奋斗！
alert [@] too long   ：昨天傍晚洗澡回来的路上，有一对情侣问我近春园宾馆怎么走，我给他们指向的目的地是@清华大学图书馆。希望他们能在知识的海洋中找回迷路的自己，为中华民族的崛起努力奋斗！
alert [@] too long @温军【终国的“戈培尔”】+此人是谁？清华大学新闻传播学院副院长李希光！因何出名？他有两个“建议”。一、人大应立法禁止网络匿名发表自己的言论；二、建议人大立法禁止个人自由思维，对个人思维立法干涉！转发！请人肉他个王八蛋，查一下他的财产。
alert [@] too long 【报告：2012年中国经济增长率有望下降到8．4％左右】清华大学中国与世界经济研究中心发布研究报告预计，2012年中国全年经济增长率有望下降到8．4％左右。央行货币委员会委员@李稻葵认为，2012年是中国经济退出刺激性扩张、回归自发增长的关键性一年，中国经济软着陆有望全面实现。
alert [转@] too long 转@饭热词英国《泰晤士报高等教育副刊》10月6日发布2011-2012世界大学排行榜，美国加州理工学院出人意料地荣登榜首，哈佛大学8年来首次失去头把交椅，与斯坦福大学并列第二。中国的北京大学和清华大学跻身百强，分列第49和第71位，中科大排名第192，香港大学位列第34。
alert [@] too long 庆国庆，微软特别推出免费赠送大礼包活动，如您是大学生或您身边的朋友是大学生，请将其姓名和所在高校信息如（张三+清华大学）发到活动官方邮箱dalibao@shop.edu.cn，工作人员会将价值970元的大礼包发送到您邮箱中。 @xiaoyuanxianfeng
alert [@] too long 庆国庆，微软特别推出免费赠送大礼包活

100%|██████████| 10031/10031 [00:00<00:00, 10619.25it/s]


In [154]:
print(len(sentences))

20527


In [155]:
# 对sentences去重。。 拒绝转发的影响 （存疑？）
# 不去重

In [156]:
# sentences = list(set(sentences))

In [157]:
print(len(list(set(sentences))))

15396


In [158]:
# torch.save(['尼玛', '卧槽', '傻逼', '对了', '有木有', '他妈'],'blacklist.torch.pkl' )
def add_to_blacklist(w):
    global blacklist
    blacklist = torch.load('blacklist.torch.pkl')
    blacklist.append(w)
    torch.save(blacklist,'blacklist.torch.pkl')

In [159]:
# 无意义
blacklist = torch.load('blacklist.torch.pkl')

In [160]:
words_all = []

segmentor = Segmentor()
postagger = Postagger()
recognizer = NamedEntityRecognizer()
postagger.load('./ltp_data_v3.4.0/pos.model')
segmentor.load('./ltp_data_v3.4.0/cws.model')
recognizer.load('./ltp_data_v3.4.0/ner.model')

#todo 情感分析

for s in tqdm(sentences):
    words = segmentor.segment(s)
    postags = postagger.postag(words) #词性标注
    netags = recognizer.recognize(words, postags) # 命名实体识别
    for i in range(len(words)):
        if words[i] in blacklist:
            continue
        words_all.append((words[i],postags[i],netags[i]))

segmentor.release()
postagger.release()
recognizer.release()

100%|██████████| 20527/20527 [00:23<00:00, 873.56it/s] 


In [161]:
print(len(words_all))

263183


In [173]:
get_top([x[0] for x in words_all if len(x[0])>1 and x[1]!='wp'], num = 25)

[('清华大学', 10370, 0.07969015361679564),
 ('大学', 1259, 0.009675014793013087),
 ('中国', 1063, 0.008168817096880788),
 ('清华', 1062, 0.008161132414757664),
 ('餐厅', 887, 0.006816313043210968),
 ('学院', 655, 0.005033466790646205),
 ('图书馆', 618, 0.004749133552090618),
 ('紫荆园', 613, 0.004710710141474998),
 ('一个', 525, 0.004034458114640088),
 ('教授', 519, 0.003988350021901344),
 ('荷塘', 499, 0.0038346563794388645),
 ('月色', 498, 0.0038269716973157405),
 ('学生', 475, 0.003650224008483889),
 ('食堂', 467, 0.003588746551498897),
 ('超级大国', 431, 0.003312097995066434),
 ('北京', 388, 0.002981656663772103),
 ('社会', 366, 0.0028125936570633754),
 ('教学楼', 365, 0.0028049089749402514),
 ('西门', 357, 0.0027434315179552596),
 ('我们', 277, 0.0021286569481053417),
 ('没有', 273, 0.0020979182196128455),
 ('第一', 273, 0.0020979182196128455),
 ('这个', 262, 0.002013386716258482),
 ('美国', 258, 0.001982647987765986),
 ('北京大学', 253, 0.0019442245771503662)]

In [163]:
posdict = {}
nedict = {}
for x in tqdm(words_all):
    if x[1] not in posdict:
        posdict[x[1]] = []
    if x[2] not in nedict:
        nedict[x[2]] = []
    posdict[x[1]].append(x)
    nedict[x[2]].append(x)

100%|██████████| 263183/263183 [00:00<00:00, 627198.23it/s]


In [164]:
# 词性
print(posdict.keys())


dict_keys(['ws', 'nt', 'r', 'v', 'j', 'u', 'n', 'd', 'm', 'q', 'wp', 'ni', 'nh', 'ns', 'i', 'nd', 'a', 'b', 'c', 'p', 'nl', 'o', 'nz', 'z', 'e', 'k', 'h', '%'])


In [174]:
# ['i'] 成语
pprint(get_top(posdict['i'], num=25))

[('高等教育', 24, 0.02185792349726776),
 ('长篇大论', 19, 0.017304189435336976),
 ('吐槽', 16, 0.014571948998178506),
 ('差不多', 15, 0.01366120218579235),
 ('自强不息', 14, 0.012750455373406194),
 ('水', 14, 0.012750455373406194),
 ('语重心长', 13, 0.011839708561020037),
 ('操作系统', 12, 0.01092896174863388),
 ('有意思', 12, 0.01092896174863388),
 ('近年来', 12, 0.01092896174863388),
 ('脱颖而出', 11, 0.010018214936247723),
 ('家庭教育', 11, 0.010018214936247723),
 ('他妈的', 10, 0.009107468123861567),
 ('如临大敌', 9, 0.00819672131147541),
 ('寅吃卯粮', 8, 0.007285974499089253),
 ('抱残守缺', 8, 0.007285974499089253),
 ('伤不起', 8, 0.007285974499089253),
 ('社会科学', 7, 0.006375227686703097),
 ('没什么', 7, 0.006375227686703097),
 ('计划生育', 6, 0.00546448087431694),
 ('逍遥法外', 6, 0.00546448087431694),
 ('泣不成声', 6, 0.00546448087431694),
 ('不堪回首', 6, 0.00546448087431694),
 ('正儿八经', 6, 0.00546448087431694),
 ('你好', 6, 0.00546448087431694)]


In [113]:

# ['j'] 缩略词
pprint(get_top(posdict['j'], num=25))


[(('清华', 'j', 'O'), 730, 0.34096216721158334),
 (('北大', 'j', 'O'), 138, 0.06445586174684727),
 (('高校', 'j', 'O'), 113, 0.052779075198505374),
 (('社科', 'j', 'O'), 92, 0.04297057449789818),
 (('经管', 'j', 'I-Ni'), 43, 0.02008407286314806),
 (('美', 'j', 'S-Ns'), 40, 0.018682858477347034),
 (('清华', 'j', 'B-Ni'), 33, 0.015413358243811303),
 (('京', 'j', 'S-Ns'), 23, 0.010742643624474545),
 (('人大', 'j', 'O'), 22, 0.010275572162540868),
 (('央视', 'j', 'O'), 20, 0.009341429238673517),
 (('东大', 'j', 'E-Ni'), 20, 0.009341429238673517),
 (('社保', 'j', 'O'), 18, 0.008407286314806165),
 (('环保', 'j', 'O'), 15, 0.007006071929005138),
 (('微博', 'j', 'O'), 15, 0.007006071929005138),
 (('中', 'j', 'S-Ns'), 14, 0.006539000467071462),
 (('经管', 'j', 'O'), 13, 0.006071929005137786),
 (('国企', 'j', 'O'), 13, 0.006071929005137786),
 (('中科院', 'j', 'S-Ni'), 13, 0.006071929005137786),
 (('欧', 'j', 'S-Ns'), 12, 0.005604857543204111),
 (('政法', 'j', 'I-Ni'), 12, 0.005604857543204111),
 (('中科院', 'j', 'O'), 12, 0.0056048575

In [114]:

# ['n'] 名词
pprint(get_top(posdict['n'], num=25))


[(('餐厅', 'n', 'O'), 787, 0.02005095541401274),
 (('人', 'n', 'O'), 691, 0.017605095541401272),
 (('大学', 'n', 'E-Ni'), 572, 0.014573248407643312),
 (('紫荆园', 'n', 'O'), 562, 0.014318471337579618),
 (('学院', 'n', 'E-Ni'), 409, 0.010420382165605096),
 (('楼', 'n', 'O'), 399, 0.0101656050955414),
 (('教授', 'n', 'O'), 393, 0.010012738853503185),
 (('食堂', 'n', 'O'), 380, 0.009681528662420382),
 (('学生', 'n', 'O'), 324, 0.008254777070063694),
 (('大学', 'n', 'O'), 295, 0.007515923566878981),
 (('教学楼', 'n', 'O'), 279, 0.00710828025477707),
 (('社会', 'n', 'O'), 194, 0.004942675159235669),
 (('紫荆', 'n', 'O'), 185, 0.004713375796178344),
 (('清芬园', 'n', 'O'), 178, 0.004535031847133758),
 (('小时', 'n', 'O'), 164, 0.004178343949044586),
 (('图书馆', 'n', 'E-Ni'), 163, 0.004152866242038217),
 (('图书馆', 'n', 'O'), 157, 0.004),
 (('西门', 'n', 'O'), 153, 0.0038980891719745223),
 (('时', 'n', 'O'), 149, 0.0037961783439490445),
 (('博士', 'n', 'O'), 147, 0.003745222929936306),
 (('老师', 'n', 'O'), 146, 0.0037197452229299362

In [115]:

# ['nh'] 人名词
pprint(get_top(posdict['nh'], num=25))


[(('王兴', 'nh', 'S-Nh'), 39, 0.014059120403749098),
 (('李稻葵', 'nh', 'S-Nh'), 36, 0.012977649603460706),
 (('孙立平', 'nh', 'S-Nh'), 34, 0.012256669069935111),
 (('蔡继明', 'nh', 'S-Nh'), 26, 0.009372746935832732),
 (('阎学通', 'nh', 'S-Nh'), 25, 0.009012256669069935),
 (('杨燕绥', 'nh', 'S-Nh'), 23, 0.00829127613554434),
 (('蒋介石', 'nh', 'S-Nh'), 23, 0.00829127613554434),
 (('朱令', 'nh', 'S-Nh'), 23, 0.00829127613554434),
 (('胡', 'nh', 'S-Nh'), 22, 0.007930785868781542),
 (('蒙民伟', 'nh', 'S-Nh'), 19, 0.00684931506849315),
 (('秦晖', 'nh', 'S-Nh'), 19, 0.00684931506849315),
 (('王', 'nh', 'S-Nh'), 19, 0.00684931506849315),
 (('梅贻琦', 'nh', 'S-Nh'), 18, 0.006488824801730353),
 (('杨振宁', 'nh', 'S-Nh'), 18, 0.006488824801730353),
 (('春娇', 'nh', 'S-Nh'), 17, 0.0061283345349675555),
 (('吴伟民', 'nh', 'S-Nh'), 16, 0.005767844268204758),
 (('白重恩', 'nh', 'S-Nh'), 16, 0.005767844268204758),
 (('李希光', 'nh', 'S-Nh'), 16, 0.005767844268204758),
 (('陈寅恪', 'nh', 'S-Nh'), 15, 0.005407354001441961),
 (('扎克伯格', 'nh', 'S-Nh'),

In [116]:

# ['ni'] 组织名
pprint(get_top(posdict['ni'], num=25))


[(('清华大学', 'ni', 'S-Ni'), 5860, 0.8070513703346647),
 (('清华大学', 'ni', 'B-Ni'), 927, 0.1276683652389478),
 (('北京大学', 'ni', 'S-Ni'), 171, 0.02355047514116513),
 (('清华大学', 'ni', 'E-Ni'), 100, 0.013772207684891888),
 (('教育部', 'ni', 'S-Ni'), 30, 0.0041316623054675665),
 (('清华大学', 'ni', 'I-Ni'), 25, 0.003443051921222972),
 (('中山大学', 'ni', 'S-Ni'), 24, 0.003305329844374053),
 (('国务院', 'ni', 'S-Ni'), 19, 0.002616719460129459),
 (('北京大学', 'ni', 'B-Ni'), 18, 0.00247899738328054),
 (('清华大学', 'ni', 'O'), 11, 0.0015149428453381077),
 (('微软', 'ni', 'S-Ni'), 9, 0.00123949869164027),
 (('中共中央', 'ni', 'S-Ni'), 7, 0.0009640545379424321),
 (('中共中央', 'ni', 'B-Ni'), 6, 0.0008263324610935132),
 (('外交部', 'ni', 'S-Ni'), 5, 0.0006886103842445945),
 (('国务院', 'ni', 'B-Ni'), 5, 0.0006886103842445945),
 (('北京大学', 'ni', 'I-Ni'), 3, 0.0004131662305467566),
 (('教育部', 'ni', 'O'), 3, 0.0004131662305467566),
 (('清华大学', 'ni', 'B-Ns'), 3, 0.0004131662305467566),
 (('文化部', 'ni', 'S-Ni'), 2, 0.0002754441536978378),
 (('团中央'

In [117]:

# ['nz'] other proper name
pprint(get_top(posdict['nz'], num=25))


[(('英语', 'nz', 'O'), 35, 0.05443234836702955),
 (('复旦', 'nz', 'B-Ni'), 30, 0.04665629860031104),
 (('百度', 'nz', 'O'), 22, 0.03421461897356143),
 (('中文', 'nz', 'O'), 20, 0.03110419906687403),
 (('维基百科', 'nz', 'O'), 20, 0.03110419906687403),
 (('新浪', 'nz', 'O'), 19, 0.029548989113530325),
 (('腾讯', 'nz', 'O'), 16, 0.024883359253499222),
 (('雄安', 'nz', 'O'), 14, 0.02177293934681182),
 (('协和', 'nz', 'O'), 13, 0.02021772939346812),
 (('复旦', 'nz', 'O'), 11, 0.017107309486780714),
 (('中华', 'nz', 'O'), 11, 0.017107309486780714),
 (('同济', 'nz', 'B-Ni'), 9, 0.013996889580093312),
 (('英文', 'nz', 'O'), 9, 0.013996889580093312),
 (('协和', 'nz', 'I-Ns'), 9, 0.013996889580093312),
 (('中文', 'nz', 'I-Ni'), 8, 0.012441679626749611),
 (('西语', 'nz', 'O'), 8, 0.012441679626749611),
 (('人民日报', 'nz', 'O'), 8, 0.012441679626749611),
 (('汉语', 'nz', 'O'), 8, 0.012441679626749611),
 (('日语', 'nz', 'O'), 7, 0.01088646967340591),
 (('西门', 'nz', 'O'), 5, 0.007776049766718507),
 (('雄安', 'nz', 'B-Ni'), 5, 0.007776049766

In [118]:

# ['v'] verb len >= 2
pprint(get_top([x for x in posdict['v'] if len(x[0])>=2], num=25))


[(('毕业', 'v', 'O'), 206, 0.010699075516775736),
 (('可以', 'v', 'O'), 149, 0.007738651708735847),
 (('知道', 'v', 'O'), 131, 0.006803781032512724),
 (('研究', 'v', 'O'), 126, 0.006544094733561857),
 (('没有', 'v', 'O'), 122, 0.0063363456944011635),
 (('工作', 'v', 'O'), 118, 0.0061285966552404696),
 (('表示', 'v', 'O'), 117, 0.006076659395450296),
 (('活动', 'v', 'O'), 117, 0.006076659395450296),
 (('改革', 'v', 'O'), 111, 0.005765035836709256),
 (('分享', 'v', 'O'), 111, 0.005765035836709256),
 (('看到', 'v', 'O'), 104, 0.005401475018178041),
 (('觉得', 'v', 'O'), 101, 0.005245663238807521),
 (('发现', 'v', 'O'), 100, 0.005193725979017347),
 (('开始', 'v', 'O'), 99, 0.005141788719227174),
 (('发布', 'v', 'O'), 96, 0.004985976939856653),
 (('学习', 'v', 'O'), 94, 0.004882102420276306),
 (('认为', 'v', 'O'), 93, 0.004830165160486133),
 (('发展', 'v', 'O'), 91, 0.004726290640905786),
 (('教育', 'v', 'O'), 85, 0.004414667082164745),
 (('退休', 'v', 'O'), 83, 0.004310792562584398),
 (('排名', 'v', 'O'), 82, 0.004258855302794224)

In [120]:

# foreign words	
pprint(get_top([x for x in posdict['ws'] if len(x[0])>=4], num=25))


[(('Facebook', 'ws', 'O'), 26, 0.003075103489059728),
 (('EMBA', 'ws', 'O'), 25, 0.0029568302779420462),
 (('http', 'ws', 'O'), 19, 0.0022471910112359553),
 (('content', 'ws', 'O'), 12, 0.0014192785334121822),
 (('tooltip', 'ws', 'O'), 12, 0.0014192785334121822),
 (('ISBN', 'ws', 'O'), 12, 0.0014192785334121822),
 (('emba', 'ws', 'O'), 11, 0.0013010053222945004),
 (('smth', 'ws', 'O'), 9, 0.0010644589000591367),
 (('ASC13', 'ws', 'O'), 7, 0.0008279124778237729),
 (('http://i.vistart.name/blog/', 'ws', 'O'), 7, 0.0008279124778237729),
 (('Tsinghua', 'ws', 'O'), 7, 0.0008279124778237729),
 (('better', 'ws', 'O'), 6, 0.0007096392667060911),
 (('http://t.cn/hbozB2', 'ws', 'O'), 6, 0.0007096392667060911),
 (('Linux', 'ws', 'O'), 6, 0.0007096392667060911),
 (('GLUT1', 'ws', 'O'), 6, 0.0007096392667060911),
 (('ＬＯＳＥＲ', 'ws', 'O'), 6, 0.0007096392667060911),
 (('http://k.ai/poi/4305311', 'ws', 'O'), 6, 0.0007096392667060911),
 (('IPv6', 'ws', 'O'), 6, 0.0007096392667060911),
 (('http://fan.fo/

In [121]:
# 命名实体
print(nedict.keys())


dict_keys(['O', 'S-Ni', 'B-Ni', 'E-Ni', 'I-Ni', 'S-Nh', 'S-Ns', 'B-Nh', 'E-Nh', 'B-Ns', 'I-Ns', 'E-Ns', 'I-Nh'])


In [122]:


# 人名
pprint(get_top(nedict['S-Nh'], num=25))


[(('王兴', 'nh', 'S-Nh'), 39, 0.015186915887850467),
 (('李稻葵', 'nh', 'S-Nh'), 36, 0.014018691588785047),
 (('孙立平', 'nh', 'S-Nh'), 34, 0.0132398753894081),
 (('蔡继明', 'nh', 'S-Nh'), 26, 0.010124610591900311),
 (('阎学通', 'nh', 'S-Nh'), 25, 0.009735202492211837),
 (('杨燕绥', 'nh', 'S-Nh'), 23, 0.00895638629283489),
 (('蒋介石', 'nh', 'S-Nh'), 23, 0.00895638629283489),
 (('朱令', 'nh', 'S-Nh'), 23, 0.00895638629283489),
 (('胡', 'nh', 'S-Nh'), 22, 0.008566978193146417),
 (('蒙民伟', 'nh', 'S-Nh'), 19, 0.007398753894080997),
 (('秦晖', 'nh', 'S-Nh'), 19, 0.007398753894080997),
 (('王', 'nh', 'S-Nh'), 19, 0.007398753894080997),
 (('梅贻琦', 'nh', 'S-Nh'), 18, 0.007009345794392523),
 (('杨振宁', 'nh', 'S-Nh'), 18, 0.007009345794392523),
 (('春娇', 'nh', 'S-Nh'), 17, 0.00661993769470405),
 (('吴伟民', 'nh', 'S-Nh'), 16, 0.006230529595015576),
 (('白重恩', 'nh', 'S-Nh'), 16, 0.006230529595015576),
 (('李希光', 'nh', 'S-Nh'), 16, 0.006230529595015576),
 (('陈寅恪', 'nh', 'S-Nh'), 15, 0.005841121495327103),
 (('扎克伯格', 'nh', 'S-Nh'), 

In [123]:

# 机构名
pprint(get_top(nedict['S-Ni'], num=25))


[(('清华大学', 'ni', 'S-Ni'), 5860, 0.9516076648262423),
 (('北京大学', 'ni', 'S-Ni'), 171, 0.027768756089639492),
 (('教育部', 'ni', 'S-Ni'), 30, 0.004871711594673595),
 (('中山大学', 'ni', 'S-Ni'), 24, 0.003897369275738876),
 (('国务院', 'ni', 'S-Ni'), 19, 0.003085417343293277),
 (('中科院', 'j', 'S-Ni'), 13, 0.002111075024358558),
 (('微软', 'ni', 'S-Ni'), 9, 0.0014615134784020786),
 (('中共中央', 'ni', 'S-Ni'), 7, 0.0011367327054238388),
 (('外交部', 'ni', 'S-Ni'), 5, 0.0008119519324455992),
 (('文化部', 'ni', 'S-Ni'), 2, 0.0003247807729782397),
 (('团中央', 'ni', 'S-Ni'), 2, 0.0003247807729782397),
 (('统战部', 'ni', 'S-Ni'), 1, 0.00016239038648911984),
 (('新华社', 'ni', 'S-Ni'), 1, 0.00016239038648911984),
 (('中国队', 'ni', 'S-Ni'), 1, 0.00016239038648911984),
 (('党中央', 'ni', 'S-Ni'), 1, 0.00016239038648911984),
 (('无线队', 'ni', 'S-Ni'), 1, 0.00016239038648911984),
 (('响应人社部', 'ni', 'S-Ni'), 1, 0.00016239038648911984),
 (('日本队', 'ni', 'S-Ni'), 1, 0.00016239038648911984),
 (('公安部', 'ni', 'S-Ni'), 1, 0.00016239038648911984),

In [124]:

# 地名
pprint(get_top(nedict['S-Ns'], num=25))

[(('中国', 'ns', 'S-Ns'), 618, 0.2727272727272727),
 (('美国', 'ns', 'S-Ns'), 159, 0.07016769638128861),
 (('北京', 'ns', 'S-Ns'), 143, 0.06310679611650485),
 (('台湾', 'ns', 'S-Ns'), 52, 0.02294792586054722),
 (('深圳', 'ns', 'S-Ns'), 41, 0.018093556928508385),
 (('美', 'j', 'S-Ns'), 40, 0.0176522506619594),
 (('日本', 'ns', 'S-Ns'), 35, 0.015445719329214475),
 (('清芬园', 'ns', 'S-Ns'), 30, 0.01323918799646955),
 (('中关村', 'ns', 'S-Ns'), 29, 0.012797881729920565),
 (('香港', 'ns', 'S-Ns'), 26, 0.01147396293027361),
 (('亚洲', 'ns', 'S-Ns'), 25, 0.011032656663724626),
 (('龙泉寺', 'ns', 'S-Ns'), 24, 0.01059135039717564),
 (('南京', 'ns', 'S-Ns'), 24, 0.01059135039717564),
 (('京', 'j', 'S-Ns'), 23, 0.010150044130626656),
 (('英国', 'ns', 'S-Ns'), 21, 0.009267431597528684),
 (('闻馨园', 'ns', 'S-Ns'), 21, 0.009267431597528684),
 (('文北', 'ns', 'S-Ns'), 20, 0.0088261253309797),
 (('五道口', 'ns', 'S-Ns'), 17, 0.007502206531332745),
 (('东门店', 'ns', 'S-Ns'), 16, 0.00706090026478376),
 (('东主楼', 'ns', 'S-Ns'), 15, 0.006619593

In [100]:
# 以上内容均为自动生成...